In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read.format("csv").option("header",True).option("inferSchema",True).load("RCdata/rating_final.csv")
data.show()

+------+-------+------+-----------+--------------+
|userID|placeID|rating|food_rating|service_rating|
+------+-------+------+-----------+--------------+
|     1| 135085|     2|          2|             2|
|     2| 135038|     2|          2|             1|
|     3| 132825|     2|          2|             2|
|     4| 135060|     1|          2|             2|
|     5| 135104|     1|          1|             2|
|     6| 132740|     0|          0|             0|
|     7| 132663|     1|          1|             1|
|     8| 132732|     0|          0|             0|
|     9| 132630|     1|          1|             1|
|    10| 132584|     2|          2|             2|
|    11| 132733|     1|          1|             1|
|    12| 132732|     1|          2|             2|
|    13| 132630|     1|          0|             1|
|    14| 135104|     0|          0|             0|
|    15| 132560|     1|          0|             0|
|    16| 132584|     1|          2|             1|
|    17| 132732|     0|        

In [4]:
(training, test) = data.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True,userCol="userID", itemCol="placeID", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [7]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
#rmse = evaluator.evaluate(predictions)
#print("Root-mean-square error = " + str(rmse))
userRecs = model.recommendForAllUsers(10)
movieRecs = model.recommendForAllItems(10)
userRecs.show(5,False)

+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userID|recommendations                                                                                                                                                                                                         |
+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|463   |[[132862,0.48100093], [135060,0.36393434], [135032,0.25766575], [132754,0.18146174], [135051,0.17885107], [135072,0.1247983], [132723,0.12071887], [135057,0.104022026], [132872,0.102057114], [135058,0.09104104]]     |
|833   |[[135038,0.09330819], [132825,0.090056315], [135039,0.05923301], [135057,0.057779774], [